In [2]:
import numpy as np
import torch
import cv2
#from pathlib import Path
import time

grid_size = 6
grid_center = int(grid_size/2)
grid_spacing = 50 # adjust as needed



# Camera Settings
camera_Width  = 1024 # 1280 # 640
camera_Heigth = 780  # 960  # 480
centerZone    = 100

# GridLine color green and thickness
lineColor = (0, 255, 0) 
lineThickness = 1

# message color and thickness
colorWhite = (255,255,255)
colorBlack = (0,0,0)
colorBlue = (255, 0, 0) 
colorGreen = (0, 255, 0) 
colorRed = (0, 0, 255) #red
messageThickness = 2

dsize = (camera_Width, camera_Heigth)

def displayGrid(frame):
    # Add a 5x5 Grid

    for i in range(grid_size):
        cv2.line(frame, (int(camera_Width/grid_size)*i, 0), (int(camera_Width/grid_size)*i, camera_Heigth), lineColor, lineThickness)
        cv2.line(frame, (0, int(camera_Heigth/grid_size)*i), (camera_Width, int(camera_Heigth/grid_size)*i), lineColor, lineThickness)
    
    # Add center lines
    cv2.line(frame, (int(camera_Width/2), 0), (int(camera_Width/2), camera_Heigth), lineColor, lineThickness)
    cv2.line(frame, (0, int(camera_Heigth/2)), (camera_Width, int(camera_Heigth/2)), lineColor, lineThickness)

    
def calculatePositionForDetectedPerson(frame, x, y, h, w):

    # calculate direction and relative position of the person
    cx = int(x + (w / 2))  # Center X of the person
    cy = int(y + (h / 2))  # Center Y of the person
    dir_list = []

    if (cx < int(camera_Width/2)-grid_spacing*grid_center):
        if (cx < int(camera_Width/2)-2*grid_spacing*grid_center):
            dir_list.append("GO FAR LEFT ")
        else:
            dir_list.append("GO LEFT ")
    elif (cx > int(camera_Width/2)+grid_spacing*grid_center):
        if (cx > int(camera_Width/2)+2*grid_spacing*grid_center):
            dir_list.append("GO FAR RIGHT ")
        else:
            dir_list.append("GO RIGHT ")
        
    if (cy < int(camera_Heigth/2)-grid_spacing*grid_center):
        if (cy < int(camera_Heigth/2)-2*grid_spacing*grid_center):
            dir_list.append("GO FAR UP ")
        else:
            dir_list.append("GO UP ")
    elif (cy > int(camera_Heigth/2)+grid_spacing*grid_center):
        if (cy > int(camera_Heigth/2)+2*grid_spacing*grid_center):
            dir_list.append("GO FAR DOWN ")
        else:
            dir_list.append("GO DOWN ")
            
    if not dir_list:
        cv2.putText(frame, "CENTER", (20, 50), cv2.FONT_HERSHEY_COMPLEX, 1, colorRed, 2)
    else:
        cv2.putText(frame, " ".join(dir_list), (20, 50), cv2.FONT_HERSHEY_COMPLEX, 1, colorRed, 2)
        
    # display detected person frame, line from center and direction to go
    cv2.line(frame, (int(camera_Width/2), int(camera_Heigth/2)), (cx, cy), colorWhite, messageThickness)
    cv2.rectangle(frame, (x, y), (x + w, y + h), colorRed, messageThickness)
    cv2.putText(frame, str(int(x)) + " " + str(int(y)), (x - 20, y - 45), cv2.FONT_HERSHEY_COMPLEX, 0.7, colorRed, messageThickness)



video_capture=cv2.VideoCapture('yürüyeninsan.mp4')
time.sleep(1.0)

model = torch.hub.load('ultralytics/yolov5', 'yolov5n', pretrained=True)


while True:
    
    ret,frameOrig =video_capture.read()
    if not ret:
        break
        

    frame=cv2.resize(frameOrig, dsize)
    displayGrid(frame)

    results = model(frame)

    persons = model.names[0] 

    person_counter = 0

    person_id = 0
   
    df = results.pandas().xyxy[0]  # get the DataFrame of detected objects
    person_boxes = {}  # dictionary to store the box with highest confidence for each person
    for _, row in df.iterrows():
        if row['name'] == persons:  # check if the detected object is a person
            if 'person_id' not in row:  # if person_id is not assigned yet
                row['person_id'] = person_counter  # assign a new person_id
                person_counter += 1
            else:
                person_id = row['person_id']  # use the assigned person_id
            if person_id not in person_boxes or row['confidence'] > person_boxes[person_id]['confidence']:
                person_boxes[person_id] = row

    # draw the box with highest confidence for each person
    person_id = 0  # initialize person ID counter

    for person_id, box in person_boxes.items():
        x, y, w, h = int(box['xmin']), int(box['ymin']), int(box['xmax'] - box['xmin']), int(box['ymax'] - box['ymin'])
        calculatePositionForDetectedPerson(frame, x, y, h, w)
        cv2.putText(frame, f'Person {person_id}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        person_id += 1
            #cv2.putText(frame, f'Persons: {total_persons}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Human Detection",frame)
    if cv2.waitKey(1)&0xFF == ord('q'):
        break
        
video_capture.release()
cv2.destroyAllWindows()

Using cache found in C:\Users\mrvbu/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-2-2 Python-3.9.12 torch-1.13.1 CUDA:0 (NVIDIA GeForce MX110, 2048MiB)

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients
Adding AutoShape... 
